In [1]:
import pandas as pd
import numpy as np
import string
import random as rn

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

from IPython.display import SVG
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import scipy.sparse as sparse
import pickle
import csv
import implicit
import itertools
import copy
plt.style.use('ggplot')

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge, Multiply,Concatenate, Bidirectional, GlobalMaxPool1D, Dense, Flatten, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, SpatialDropout1D
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model, save_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping

from keras.layers.merge import dot


Using TensorFlow backend.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
path = '/Users/xun/Documents/Thesis/'
# path = '/content/drive/My Drive/Thesis_share/'
# path = '/Users/xing/Google Drive/Thesis_share/'

In [4]:
#get recipe data
recipe_all = recipe_all = pd.read_csv(path + 'recipe_data/recipe_data_processed.csv') 
recipes = recipe_all.iloc[:,0:3]
recipes['publishdate'] = recipes['publishdate'].astype('datetime64')
recipes["pub_date_idx_grp"] = recipes["publishdate"].rank(method='max')
recipes["pub_date_idx"] = recipes["publishdate"].rank(method='first')
# recipes['rid'] = recipes.index.astype('int')
recipes = recipes.sort_values(by=['pub_date_idx'])

# recipes.head(10)

# # Order by publish date so it can be used as a filtering index for set comparisons
# recipes_list = list(recipes['rid'].values)
# recipes_list[:10]

recipes_dates = list(recipes['publishdate'].values)

,publishdate,content_id,title_en,pub_date_idx_grp,pub_date_idx,rid
12,2014-07-15,R70,Pull-apart bread,59.0,1.0,12
17,2014-07-15,R15,Chicken fajitas,59.0,2.0,17
34,2014-07-15,R17,Caesar salad,59.0,3.0,34
74,2014-07-15,R32,Double chocolate tart,59.0,4.0,74
85,2014-07-15,R44,Fudgy brownies with pecans,59.0,5.0,85
129,2014-07-15,R80,Vegan Sloppy Joe burger,59.0,6.0,129
152,2014-07-15,R13,Chicken wings,59.0,7.0,152
184,2014-07-15,R5,Blueberry muffins,59.0,8.0,184
206,2014-07-15,R2,Angel food cake with fresh strawberries,59.0,9.0,206
319,2014-07-15,R91,Turbot a la Bordelaise with Swiss chard,59.0,10.0,319


[12, 17, 34, 74, 85, 129, 152, 184, 206, 319]

In [47]:
recipes_list

[12,
 17,
 34,
 74,
 85,
 129,
 152,
 184,
 206,
 319,
 357,
 359,
 367,
 370,
 376,
 381,
 382,
 383,
 399,
 400,
 408,
 516,
 519,
 611,
 620,
 628,
 631,
 640,
 642,
 662,
 754,
 755,
 781,
 843,
 861,
 891,
 892,
 899,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 1587,
 1591,
 1660,
 1784,
 1789,
 67,
 63,
 405,
 529,
 925,
 1761,
 804,
 1757,
 1800,
 262,
 537,
 820,
 926,
 710,
 713,
 1712,
 52,
 321,
 927,
 706,
 928,
 1661,
 816,
 809,
 929,
 930,
 233,
 238,
 252,
 387,
 390,
 443,
 860,
 931,
 1597,
 1598,
 1618,
 1762,
 932,
 796,
 40,
 793,
 791,
 454,
 59,
 458,
 68,
 385,
 933,
 1760,
 466,
 300,
 862,
 934,
 102,
 1759,
 137,
 335,
 479,
 139,
 310,
 482,
 397,
 1796,
 353,
 851,
 489,
 490,
 649,
 1798,
 13,
 364,
 496,
 935,
 936,
 497,
 742,
 735,
 724,
 723,
 278,
 282,
 320,
 504,
 505,
 207,
 722,
 937,
 183,
 386,
 728,
 835,
 938,
 939,
 509,
 940,
 941,
 942,
 284,
 577,
 747,
 943,
 753,
 878,
 944,
 1768,
 94

In [5]:
def map_ids(row, mapper):
    return mapper[row]

In [10]:
df_interactions = pd.read_csv(path + 'interaction_data/ks-18m-cooks-v1.csv')
df_interactions = df_interactions.sort_values(by = ['USERID', 'TIMESTAMP'])
df_interactions = df_interactions.reset_index(drop=True)

df_i = df_interactions.iloc[:, 0:3]
df_i.columns = ['USERID', 'content_id', 'TIMESTAMP']
df_i.TIMESTAMP = df_i.TIMESTAMP.astype('datetime64')

In [15]:
df_i = df_i.merge(recipes, 'left', on = 'content_id' )
df_i = df_i[['USERID', 'title_en', 'TIMESTAMP']]
df_i =df_i[~df_i['title_en'].isna()]

In [23]:
recipes['rid'] = recipes.title_en.apply(map_ids, args=[recipe_to_index]).values

In [17]:
# Create mappings
index_to_recipe = np.load(path + 'recipe_data/index_to_recipe.npy', allow_pickle=True).item()
recipe_to_index = np.load(path + 'recipe_data/recipe_to_index.npy', allow_pickle=True).item()
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_i.USERID.unique().tolist()):
  uid_to_idx[uid] = idx
  idx_to_uid[idx] = uid

df_i['uid'] = df_i.USERID.apply(map_ids, args=[uid_to_idx]).values
df_i['rid'] = df_i.title_en.apply(map_ids, args=[recipe_to_index]).values

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [18]:
df_i.head()

,USERID,title_en,TIMESTAMP,uid,rid
0,1735967714,Fettuccine Alfredo,2017-11-15 19:20:53.731,0,808
1,1735967714,Zurich-style veal and mushroom ragout,2017-11-28 17:41:43.973,0,570
2,1735967714,Shakshuka bowl,2017-11-29 17:54:51.629,0,1676
3,1735967714,Shakshuka,2017-11-29 17:55:28.847,0,901
4,1735967714,Thai peanut-coconut soup with rice noodles,2018-01-23 07:39:59.955,0,1795


In [19]:
def find_r_idx_corresponding_to_date(r_date):
    i = next((i for i, latest_i_date in reversed(list(enumerate(recipes_dates))) if latest_i_date < np.datetime64(r_date)), None)
    return i


In [20]:
df_u_latest_interaction = df_i.groupby(['uid'])['TIMESTAMP'].max().reset_index()
df_u_latest_interaction.columns = ['uid','latest_i_time']
df_u_latest_interaction['latest_i_r_idx'] = df_u_latest_interaction['latest_i_time'].apply(find_r_idx_corresponding_to_date)


In [21]:
df_u_latest_interaction.head()

,uid,latest_i_time,latest_i_r_idx
0,0,2019-03-30 12:23:28.520,1742
1,1,2018-12-02 19:58:34.733,1604
2,2,2019-03-21 20:10:40.293,1733
3,3,2019-02-08 21:59:36.060,1690
4,4,2019-03-14 17:42:56.002,1724


In [30]:
df_joined = df_i.merge(recipes[['rid','pub_date_idx']], 'left', on = 'rid' )
df_i_joined = df_joined.merge(df_u_latest_interaction, 'left', on = 'uid' )
# df_i_joined.head()
df_cols = df_i_joined[['USERID', 'title_en', 'TIMESTAMP', 'uid', 'rid', 'pub_date_idx','latest_i_r_idx']]
df_cols['ct'] = df_cols.groupby(['uid'])['uid'].transform('size')

df = df_cols.sort_values(by=['uid','pub_date_idx'])
df.head()
df.shape

,USERID,title_en,TIMESTAMP,uid,rid,pub_date_idx,latest_i_time,latest_i_r_idx
0,1735967714,Fettuccine Alfredo,2017-11-15 19:20:53.731,0,808,756.0,2019-03-30 12:23:28.520,1742
1,1735967714,Zurich-style veal and mushroom ragout,2017-11-28 17:41:43.973,0,570,901.0,2019-03-30 12:23:28.520,1742
2,1735967714,Shakshuka bowl,2017-11-29 17:54:51.629,0,1676,743.0,2019-03-30 12:23:28.520,1742
3,1735967714,Shakshuka,2017-11-29 17:55:28.847,0,901,287.0,2019-03-30 12:23:28.520,1742
4,1735967714,Thai peanut-coconut soup with rice noodles,2018-01-23 07:39:59.955,0,1795,820.0,2019-03-30 12:23:28.520,1742


,USERID,title_en,TIMESTAMP,uid,rid,pub_date_idx,latest_i_r_idx,ct
9,1735967714,Alsatian Tarte Flambée,2018-07-15 10:39:12.854,0,397,122.0,1742,20
19,1735967714,One-pot pasta with orzo and shrimp,2019-03-30 12:23:28.520,0,968,251.0,1742,20
3,1735967714,Shakshuka,2017-11-29 17:55:28.847,0,901,287.0,1742,20
17,1735967714,Hot chickpea soup,2019-03-20 16:21:16.883,0,883,318.0,1742,20
15,1735967714,Fish gratin,2019-01-19 17:49:20.098,0,1011,420.0,1742,20


(859880, 8)

In [31]:
# 39648 users, 1743 items, 859766 cooks
len(list(df['uid'].unique())),
len(list(df['rid'].unique()))

(39648,)

1743

In [32]:
user_input, item_input, labels = [],[],[]
num_items = 1815
num_negatives = 2
test_latest_n_interactions = 10
test_users_frac=0.1


In [33]:
df_user_test_assign = df.groupby(['uid']).apply(lambda x: np.random.choice([0,1], p=[1-test_users_frac, test_users_frac])).reset_index()
df_user_test_assign.columns = ['uid','test']

In [34]:
len(df_user_test_assign[df_user_test_assign.test ==1])

3946

In [35]:
## Returns test + train dfs with negatives labeled.  Prereq, recipes and interactions preprocessing see above ^
# def get_train_test_instances(df, num_negatives, test_sample_size): 

split_count = test_latest_n_interactions
#     df.shape

# # Filter on only users with interactions > 2 x split count
# df_filtered = df[df['ct'] >= test_latest_n_interactions]
# print(df_filtered.head())

# Column i_list interactions list grouped by USERID
df_lists_group = df.groupby(['uid', 'latest_i_r_idx', 'ct'])['rid'].apply(list).reset_index(name ='i_list')

# Join in test assignments
df_lists_tests_merge = df_lists_group.merge(df_user_test_assign, 'left', on = 'uid' )


In [36]:
# df_lists_tests_merge[test_latest_n_interactions*(1+df_lists['test']']

In [37]:
df_lists_tests_merge[df_lists_tests_merge['ct']>20].shape

(12396, 5)

In [38]:
# Filter on items where ct >= 2*latest_interaction for test users or 1*latest_interactions for training
df_filtered = df_lists_tests_merge[df_lists_tests_merge['ct'] >= test_latest_n_interactions*(1+df_lists_tests_merge['test'])]
df_lists = df_filtered

In [39]:
# df_filtered[['i_list', 'test']].apply(lambda x: x['i_list'][:int(None if x['test'] is None else -split_count)],axis=1)

In [40]:
# Split Train and test 

df_lists['train'] = df_filtered[['i_list', 'test']].apply(lambda x: x['i_list'][:None if x['test'] ==0 else -split_count],axis=1)
df_lists['test'] = df_lists[df_lists['test']==1]['i_list'].apply(lambda x: x[-split_count:])
df_lists['train_len'] = df_lists['train'].apply(lambda x: len(x))

# Column: Get list of all recipes published before latest interaction date
df_lists['pub_date_before_set'] = df_lists['latest_i_r_idx'].apply(lambda x:recipes_list[:x])

# Column: set(pub_date_before_set) - set(interaction_list)
df_lists['neg_set_full'] = df_lists[['i_list','pub_date_before_set']].apply(lambda x: set(x.pub_date_before_set)- set(x.i_list), axis= 1)

# Column: Length
df_lists['neg_set_full_length'] = df_lists[['neg_set_full', 'train_len']].apply(lambda x: len(x.neg_set_full), axis=1)

# Column: Calc generated neg set length
df_lists['neg_selected_set_length'] = df_lists[['neg_set_full', 'train_len']].apply(lambda x: min(x.train_len*num_negatives,len(x.neg_set_full)), axis=1)

# Filter out users who have seen all recipes (see Notion note)
df_lists = df_lists[df_lists['neg_selected_set_length']>=0]

# Column Randomly sample from neg set based on neg set length
df_lists['neg_selected_set'] = df_lists[['neg_set_full', 'neg_selected_set_length']].apply(lambda x: rn.sample(x.neg_set_full, x.neg_selected_set_length), axis=1)
print(df_lists.head())

# Index for the purposes of stacking later
df_lists_user_indexed = df_lists.set_index(['uid'])
print('df_lists_user_indexed')
df_lists_user_indexed.head()

# Flatten lists to DF + label
df_lists_train_stacked = df_lists_user_indexed['train'].apply(pd.Series).stack().reset_index()
df_lists_train_stacked['label'] = 1
print('df_lists_train_stacked')
df_lists_train_stacked.shape

df_lists_negs_stacked = df_lists_user_indexed['neg_selected_set'].apply(pd.Series).stack().reset_index()
df_lists_negs_stacked['label'] = 0
print('df_lists_negs_stacked')
df_lists_negs_stacked.shape

# Put the generated negs with the training
df_train_set = df_lists_negs_stacked.append(df_lists_train_stacked)
df_train_set.rename(columns={0:'rid'}, inplace=True)
print('df_train_set')
df_train_set.shape

# Flatten test set
df_test_set = df_lists_user_indexed['test'].apply(pd.Series).stack().reset_index()
df_test_set['label'] = 1
df_test_set.rename(columns={0:'rid'}, inplace=True)

    

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

   uid  latest_i_r_idx  ct                                             i_list  \
0    0            1742  20  [397, 968, 901, 883, 1011, 1676, 1673, 808, 17...   
1    1            1604  11  [604, 1008, 1743, 838, 1640, 1240, 608, 1781, ...   
2    2            1733  13  [59, 735, 724, 943, 148, 1698, 968, 503, 1174,...   
3    3            1690  16  [604, 602, 854, 591, 1014, 312, 279, 1636, 163...   
4    4            1724  70  [85, 816, 929, 59, 649, 940, 759, 947, 610, 60...   

                                                test  \
0  [783, 570, 218, 712, 695, 246, 652, 1492, 589,...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  [1417, 831, 821, 1473, 1476, 1479, 595, 1533, ...   

                                               train  train_len  \
0  [397, 968, 901, 883, 1011, 1676, 1673, 808, 17...         10   
1  [604, 1008, 1743, 838, 1640, 1240, 608,

,latest_i_r_idx,ct,i_list,test,train,train_len,pub_date_before_set,neg_set_full,neg_set_full_length,neg_selected_set_length,neg_selected_set
uid,,,,,,,,,,,
0,1742,20,"[397, 968, 901, 883, 1011, 1676, 1673, 808, 17...","[783, 570, 218, 712, 695, 246, 652, 1492, 589,...","[397, 968, 901, 883, 1011, 1676, 1673, 808, 17...",10,"[12, 17, 34, 74, 85, 129, 152, 184, 206, 319, ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1722,20,"[203, 155, 982, 477, 1451, 1009, 240, 1589, 70..."
1,1604,11,"[604, 1008, 1743, 838, 1640, 1240, 608, 1781, ...",NaN,"[604, 1008, 1743, 838, 1640, 1240, 608, 1781, ...",11,"[12, 17, 34, 74, 85, 129, 152, 184, 206, 319, ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1593,22,"[591, 351, 972, 938, 448, 1261, 425, 342, 463,..."
2,1733,13,"[59, 735, 724, 943, 148, 1698, 968, 503, 1174,...",NaN,"[59, 735, 724, 943, 148, 1698, 968, 503, 1174,...",13,"[12, 17, 34, 74, 85, 129, 152, 184, 206, 319, ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1720,26,"[512, 373, 1502, 834, 1214, 1473, 1134, 606, 1..."
3,1690,16,"[604, 602, 854, 591, 1014, 312, 279, 1636, 163...",NaN,"[604, 602, 854, 591, 1014, 312, 279, 1636, 163...",16,"[12, 17, 34, 74, 85, 129, 152, 184, 206, 319, ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1674,32,"[52, 1515, 814, 1476, 347, 974, 165, 1141, 528..."
4,1724,70,"[85, 816, 929, 59, 649, 940, 759, 947, 610, 60...","[1417, 831, 821, 1473, 1476, 1479, 595, 1533, ...","[85, 816, 929, 59, 649, 940, 759, 947, 610, 60...",60,"[12, 17, 34, 74, 85, 129, 152, 184, 206, 319, ...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1654,120,"[1356, 1718, 4, 1121, 1007, 887, 387, 1731, 50..."


df_lists_train_stacked


(810130, 4)

df_lists_negs_stacked


(1614119, 4)

df_train_set


(2424249, 4)

In [41]:
train = df_train_set[df_train_set['label']==1][['uid','rid']].applymap(int)
train.head()
train.shape
train.to_csv(path+ 'interaction_data/ks-18m-cooks-v1_train.csv')

test = df_test_set[df_test_set['label']==1][['uid','rid']].applymap(int)
test.head()
test.shape
test.to_csv(path+ 'interaction_data/ks-18m-cooks-v1_test.csv')

,uid,rid
0,0,397
1,0,968
2,0,901
3,0,883
4,0,1011


(810130, 2)

,uid,rid
0,0,783
1,0,570
2,0,218
3,0,712
4,0,695


(13250, 2)

In [42]:
len(train.uid.unique()), len(train.rid.unique())
len(test.uid.unique()), len(test.rid.unique())

(37021, 1743)

(1325, 1347)

In [43]:
I = train.uid.values
J = train.rid.values # turn mid to idx
V = np.ones(I.shape[0])
train_coo = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
train_csr = train_coo.tocsr()


I_t = test.uid.values
J_t = test.rid.values # turn mid to idx
V_t = np.ones(I_t.shape[0])
test_coo = sparse.coo_matrix((V_t, (I_t, J_t)), dtype=np.float64)
test_csr = test_coo.tocsr()

In [44]:
# Transpose results lists of lists [n x 3] > [3 x n]


# train = df_train_set[['uid','rid']].apply(int)
user_input = df_train_set['uid'].apply(int).values.tolist()
item_input = df_train_set['rid'].apply(int).values.tolist()
labels = df_train_set['label'].apply(int).values.tolist()

# df_test_set_list_of_lists = df_test_set[['uid','rid','label']].apply(int).values.tolist()
test_user_input = df_test_set['uid'].apply(int).values.tolist()
test_item_input = df_test_set['rid'].apply(int).values.tolist()
test_labels = df_test_set['label'].apply(int).values.tolist()

In [45]:
num_items = 1815
num_users = 39648
# num_negatives = 2

num_factors = 300 # number of lalent factors for user_input and itme_input
regs = [0,0]
learner = 'adam'
learning_rate = 0.001
    
topK = 10
evaluation_threads = 1

In [40]:
embedding_matrix = np.genfromtxt(path + 'recipe_data/recipe_embedding.csv', delimiter=',')
embedding_matrix.shape

(1815, 300)

In [ ]:
# model = load_model('/Users/xun/Documents/Thesis/model/cooks-18m-GMF-neg2.h5')
model = load_model(path + 'model/cooks-18m-GMF-neg_set_small_test.h5')
                   

In [0]:
def init_normal(shape, dtype=None):
    return K.random_normal(shape, dtype=dtype)

def get_model(num_users, num_items, latent_dim, trainable, regs=[0,0]):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim,
                                  input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim,
                                  input_length=1, trainable = trainable)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings 
    # predict_vector = merge([user_latent, item_latent], mode = 'mul')
    predict_vector = Multiply()([user_latent, item_latent])
    # Final prediction layer
    #prediction = Lambda(lambda x: K.sigmoid(K.sum(x)), output_shape=(1,))(predict_vector)
    prediction = Dense(1,kernel_initializer='ones')(predict_vector)
    ## prediction is the output of the Dense layer, which has 1D dimension
    model = Model(input=[user_input, item_input], 
                output=prediction)

    return model

In [42]:
# Build model
model = get_model(num_users, num_items, num_factors, False, regs)
if learner.lower() == "adagrad": 
    model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy',metrics=['accuracy'])
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
else:
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy',metrics=['accuracy'])
#print(model.summary())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [62]:
from keras.callbacks import ModelCheckpoint
model_chk_path = path + 'model'

epochs = 10
batch_size = 256
out = 1

# user_input, item_input, labels = get_train_instances(train, num_negatives)
mcp = ModelCheckpoint(model_chk_path, monitor="val_acc", verbose=1,
                      save_best_only=True, save_weights_only=False)
# Training
hist = model.fit([np.array(user_input), np.array(item_input)], #input
                 np.array(labels), # labels 
                 batch_size=batch_size, epochs=epochs, 
                 validation_split=0.1,
                 verbose=1, shuffle=True)

Train on 2183407 samples, validate on 242601 samples
Epoch 1/10
2183407/2183407 [==============================] - 97s 44us/step - loss: 0.0558 - acc: 0.9842 - val_loss: 2.3043 - val_acc: 0.2323
Epoch 2/10
2183407/2183407 [==============================] - 96s 44us/step - loss: 0.0433 - acc: 0.9872 - val_loss: 2.4440 - val_acc: 0.2498
Epoch 3/10
2183407/2183407 [==============================] - 96s 44us/step - loss: 0.0362 - acc: 0.9890 - val_loss: 2.5477 - val_acc: 0.2624
Epoch 4/10
2183407/2183407 [==============================] - 96s 44us/step - loss: 0.0318 - acc: 0.9901 - val_loss: 2.6180 - val_acc: 0.2736
Epoch 5/10
2183407/2183407 [==============================] - 96s 44us/step - loss: 0.0287 - acc: 0.9908 - val_loss: 2.6802 - val_acc: 0.2811
Epoch 6/10
2183407/2183407 [==============================] - 97s 45us/step - loss: 0.0265 - acc: 0.9913 - val_loss: 2.7395 - val_acc: 0.2862
Epoch 7/10
2183407/2183407 [==============================] - 96s 44us/step - loss: 0.0249 - ac

In [0]:
model.save(path + 'model/cooks-18m-GMF-neg_set_small_test.h5')

In [64]:
test_csr.shape

(39638, 1815)

In [0]:
def recommend(uid,item_list,user_items, model, N = 10, filter_already_liked_items=True):
    item_list.sort()
    user_items_csr = user_items.tocsr()
    num_items = len(item_list)
    user_input = [uid for i in range(num_items)]
    
    if filter_already_liked_items is True:
        liked = set(user_items_csr[uid].indices)
    else:
        liked = set()  
    scores_l = model.predict([user_input, item_list])
    scores = np.asarray([item for sublist in scores_l for item in sublist])
    count = N + len(liked)
    if count < len(scores):
        ids = np.argpartition(scores, -count)[-count:]
        best = sorted(zip(ids, scores[ids]), key=lambda x: -x[1])
    else:
        best = sorted(enumerate(scores), key=lambda x: -x[1])
    return list(itertools.islice((rec for rec in best if rec[0] not in liked), N))

In [0]:
def precision_at_k(model,test, user_items, k=10, user_index=None):
    if not user_index:
        user_index = range(len(np.unique(test.nonzero()[0])))
    user_items = user_items.tocsr()
    item_list = np.unique(user_items.nonzero()[1])
    
    precisions = []
    for user in user_index:
        rec = recommend(user,item_list,user_items, model,k, filter_already_liked_items= True)
        top_k = [rec[i][0] for i in range(len(rec))]
        labels = test.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)

In [0]:
model_precision_at_k = precision_at_k(model,test_csr,train_csr, 10)

In [68]:
model_precision_at_k

0.0002169197396963124

In [0]:
df[df.uid == 24687]
item_liked = df[df.uid == 24687].rid.tolist()[:-10]
item_test = df[df.uid == 24687].rid.tolist()[-10:]

item_liked
item_test

In [0]:
rec = recommend(24687,item_list,train, model, N = 40, filter_already_liked_items=True)

rec_df = pd.DataFrame(rec, columns = ['rid','prep'])
rec_df.merge(recipes,'left', on = 'rid' )

def get_rec_id(list):
    ids = []
    for item in list:
        item = item[0]
        ids.append(item)
    return ids
  
set(get_rec_id(rec)) & set(item_test)

In [54]:
%load_ext Cython

In [57]:
%%cython
import cython
from cython.operator import dereference
from cython.parallel import parallel, prange



In [58]:
%%cython
from libc.stdlib cimport atoi
from libc.stdlib cimport malloc, free
from libc.string cimport memset
from libc.math cimport fmin

from libcpp.unordered_set cimport unordered_set

def precision_at_k(model, train_user_items, test_user_items, K=10,
                   show_progress=True, num_threads=1):
    
    cdef users = test_user_items.shape[0], u, i
    cdef double relevant = 0, total = 0
    cdef int[:] test_indptr = test_user_items.indptr
    cdef int[:] test_indices = test_user_items.indices

    cdef int * ids
    cdef unordered_set[int] * likes

    progress = tqdm.tqdm(total=users, disable=not show_progress)

    with nogil, parallel(num_threads=num_threads):
        ids = <int *> malloc(sizeof(int) * K)
        likes = new unordered_set[int]()
        try:
            for u in prange(users, schedule='guided'):
                # if we don't have any test items, skip this user
                if test_indptr[u] == test_indptr[u+1]:
                    continue
                memset(ids, 0, sizeof(int) * K)

                with gil:
                    recs = model.recommend(u, train_user_items, N=K)
                    for i in range(len(recs)):
                        ids[i] = recs[i][0]
                    progress.update(1)

                # mostly we're going to be blocked on the gil here,
                # so try to do actual scoring without it
                likes.clear()
                for i in range(test_indptr[u], test_indptr[u+1]):
                    likes.insert(test_indices[i])

                total += fmin(K, likes.size())

                for i in range(K):
                    if likes.find(ids[i]) != likes.end():
                        relevant += 1
        finally:
            free(ids)
            del likes

    progress.close()
    return relevant / total


Error compiling Cython file:
------------------------------------------------------------
...
    cdef int[:] test_indices = test_user_items.indices

    cdef int * ids
    cdef unordered_set[int] * likes

    progress = tqdm.tqdm(total=users, disable=not show_progress)
              ^
------------------------------------------------------------

/Users/xun/.ipython/cython/_cython_magic_f799093438722866c8668fe3afa0c065.pyx:19:15: undeclared name not builtin: tqdm

Error compiling Cython file:
------------------------------------------------------------
...
    cdef int * ids
    cdef unordered_set[int] * likes

    progress = tqdm.tqdm(total=users, disable=not show_progress)

    with nogil, parallel(num_threads=num_threads):
               ^
------------------------------------------------------------

/Users/xun/.ipython/cython/_cython_magic_f799093438722866c8668fe3afa0c065.pyx:21:16: undeclared name not builtin: parallel

Error compiling Cython file:
-------------------------------

TypeError: object of type 'NoneType' has no len()